In [ ]:
import numpy as np
import simpy
import random
import scipy as sp
import matplotlib.pyplot as plt

# [Problem 1: Deriving Average Wait Times for M/M/1 vs. M/M/C Queues]

# Average Waiting Time for M/M/1 Queues 

$E(W)_{M/M/1} = \frac{E(L)_{M/M/1}}{\lambda}$ (Little's Law)

$E(L)_{M/M/1} = \Sigma{n*p_n}$

$\text{Since } p_n = (1-\rho)*\rho^n \text{, then } E(L)_{M/M/1} = \Sigma {n*(1-\rho)*\rho^n}$

$\text{Recall } \Sigma \rho^n = \frac{1}{1-\rho} \text{ by the Geometric Series}$

$\text{Then, we can differentiate both sides: } \frac{d}{d\rho} \Sigma \rho^n = \frac{d}{d\rho} \frac{1}{1-\rho} $

$\text{We get: } \Sigma n*\rho^{n-1} = \frac{1}{(1-\rho)^2}$ 

$\text{Therefore, } \Sigma n*p^n = \frac{\rho}{(1-\rho)^2}$

$\text{And } (1-\rho) * \Sigma n*\rho^n = \frac{\rho}{1-\rho}$

$\text{E(L)}_{M/M/1} = \frac{\rho}{1-\rho} = \frac{\frac{\lambda}{\mu}}{1-\rho}$

${E(W)}_{M/M/1} = \frac{{E(L)}_{M/M/1}}{\lambda} = \frac{1}{\mu*(1-\rho)}$

$\text{------------------------------------------------------------------}$

# Average Waiting Time for M/M/C Queues 
$E(W)_{M/M/C} = \frac{E(L)_{M/M/C}}{\lambda}$ (Little's Law)

$E(L)_{M/M/C} = \Sigma{n*p_{n+c}} = \Sigma{n*p^n*p_c}$

$ = p_c\Sigma{n*p^n}$

$ \text{Following logic from previous section, } \Sigma{n*p^n*p_c} = p_c * \frac{\rho}{(1-\rho)^2}$

$ = \frac{p_c}{1-\rho} * \frac{\rho}{1-\rho}$

$ \text{Since } \frac{p_c}{1-\rho} = \Pi_w \text{ and } \rho = \frac{\lambda}{n*\mu} \text{, then } E(L)_{M/M/C} = \Pi_w * \frac{\lambda}{(1-\rho)*n*\mu}$

$ \text{Therefore, } E(W)_{M/M/C} = \frac{\Pi_w}{(1-\rho)*n*\mu}$

$ \Pi_w \text{ is always <= 1 since it is the probability that a job has to wait. Therefore, } E(W)_{M/M/C} \text{ will always be lower than } E(W)_{M/M/1} \text{ by at least a factor of n.}$








# [Problem 2: DES Program to Verify Results]

In [6]:
class MMCQueueSimulation:
    def __init__(self, env, lambd, mu, n):
        '''
        Initializes object values in environment.
        Parameters:
        ----------
        lambd: Arrival rate into the system (λ)
        mu: Capacity of each server (μ)
        n: Number of servers
        '''
        self.env = env
        self.lambd = lambd
        self.mu = mu
        self.n = n
        self.servers = simpy.Resource(env, capacity=n)
        self.waiting_times = []
    
    def service_process(self):
        '''
        Process for each customer in the system. 
        '''
        arrival_time = self.env.now

        with self.servers.request() as request:
            # Wait for request
            yield request
            
            # Calculate and store waiting time after request has been approved
            waiting_time = self.env.now - arrival_time
            self.waiting_times.append(waiting_time)
            
            service_time = random.expovariate(1/self.mu)
            
            # Simulate service
            yield self.env.timeout(service_time)
    
    def arrival_process(self):
        '''
        Generates customer arrivals with exponential times between arrivals.
        '''
        while True:
            # Calculate time until arrival
            yield self.env.timeout(random.expovariate(1/self.lambd))
            
            # Continue to service process after arrival
            self.env.process(self.service_process())
    
    def simulation_run(self, simulation_time):
        self.env.process(self.arrival_process())
        self.env.run(until=simulation_time)
        avg_waiting_time = np.mean(self.waiting_times) if self.waiting_times else 0
        return avg_waiting_time


In [11]:
lambd = 9
mu = 10
simulation_time = 10000
num_simulations = 3
n_list = [1, 2, 4]  
avg_wait_times = []
var_wait_times = []

for n in n_list:
    for i in range(num_simulations):
        env = simpy.Environment()
        simulation = MMCQueueSimulation(env, lambd, mu, n)
        avg_wait_time = simulation.simulation_run(simulation_time)
        print(f'Run {i+1}')
        print(f'Number of servers (n): {n}, Average Waiting Time: {avg_wait_time: .2f}')
    print('-')

Run 1
Number of servers (n): 1, Average Waiting Time:  800.86
Run 2
Number of servers (n): 1, Average Waiting Time:  382.44
Run 3
Number of servers (n): 1, Average Waiting Time:  642.35
-
Run 1
Number of servers (n): 2, Average Waiting Time:  4.27
Run 2
Number of servers (n): 2, Average Waiting Time:  5.88
Run 3
Number of servers (n): 2, Average Waiting Time:  2.80
-
Run 1
Number of servers (n): 4, Average Waiting Time:  0.13
Run 2
Number of servers (n): 4, Average Waiting Time:  0.30
Run 3
Number of servers (n): 4, Average Waiting Time:  0.18
-
